In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/distilled_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import LSTM_Models
from compact.knowledge_distillation import KnowledgeDistillation

In [2]:
conf = Config()
load_models = LSTM_Models()
teacher_model = load_models.lstm_4
teacher_model.load()
student_model = load_models.result_light_lstm_1
kd = KnowledgeDistillation(
    teacher=teacher_model,
    student=student_model,
    device=conf.device,
    distillation="result",
    rnn=True
)

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/large_model/lstm_4.pth!


In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "lstm")

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

In [4]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)

torch.Size([32, 513, 1]) torch.Size([32])


In [5]:
acc, train_loss, val_loss = kd.train_kd(train_loader, val_loader, conf.epochs)

Epoch: 1/10, Accuracy: 88.85%, Train loss: 0.0162, Val loss: 0.1949
Epoch: 2/10, Accuracy: 91.95%, Train loss: 0.0073, Val loss: 0.1305
Epoch: 3/10, Accuracy: 92.69%, Train loss: 0.0058, Val loss: 0.1182
Epoch: 4/10, Accuracy: 93.32%, Train loss: 0.0052, Val loss: 0.1029
Epoch: 5/10, Accuracy: 93.56%, Train loss: 0.0048, Val loss: 0.0974
Epoch: 6/10, Accuracy: 93.87%, Train loss: 0.0046, Val loss: 0.0912
Epoch: 7/10, Accuracy: 94.24%, Train loss: 0.0044, Val loss: 0.0845
Epoch: 8/10, Accuracy: 94.46%, Train loss: 0.0042, Val loss: 0.0820
Epoch: 9/10, Accuracy: 94.41%, Train loss: 0.0041, Val loss: 0.0788
Epoch: 10/10, Accuracy: 94.41%, Train loss: 0.0041, Val loss: 0.0794


In [6]:
# test teacher accuracy
loss, accuracy = teacher_model.evaluate(test_loader)
print(f"Teacher accuracy: {100*accuracy:.2f}%")

Teacher accuracy: 96.64%


In [7]:
# test student accuracy
loss, accuracy = student_model.evaluate(test_loader)
print(f"Student accuracy: {100*accuracy:.2f}%")

Student accuracy: 94.39%


In [8]:
student_model.save()

Checkpoint saved at /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/compressed_model/result_light_lstm_1.pth
